In [19]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode
import requests
from bs4 import BeautifulSoup

url = 'https://main.knesset.gov.il/Activity/committees/Women/Pages/CommitteeHarassmentDocs5778.aspx'

# retrieve the content of the webpage using the requests library
response = requests.get(url)

# parse the HTML content of the webpage using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# find all the <a> tags in the HTML content
links = soup.find_all('a')

# loop through each <a> tag and print the value of the 'href' attribute
for link in links:
    href = link.get('href')
    if href is not None:
        print(href)

In [42]:
# import camelot
#
file = "https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman2.pdf"
# tables = camelot.read_pdf(file)
# for table in tables:
#     print(table.df)
import tabula
import os
links_5778 = open("links_5778",'r',encoding='utf-8')
links=links_5778.readlines()
files = [pdf for pdf in [link.split("\t")[0].replace('"',' ')[1:-1] for link in links] if pdf[-4:] == ".pdf" ][:-1]
for i,file in enumerate(files,start=1):
    tables = tabula.read_pdf(file, pages="all")
    folder_name = "5778"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    # iterate over extracted tables and export as excel individually
    for j, table in enumerate(tables, start=1):
        table.to_excel(os.path.join(folder_name, f"{i}_table_{j}.xlsx"), index=False)

In [2]:
print(tables)

[                       Unnamed: 0                          Unnamed: 1  \
0                             NaN                                 NaN   
1                             NaN                                 NaN   
2           טלפון נייח 036407875:                 מספר זהות 53293122:   
3                             NaN  tbrosh@tauex.tau.ac.il :כתובתדוא"ל   
4   אחריות: ממונה על מניעת הטרדות         שיוך במוסד: איש/ת סגל אקדמי   
5                          מיניות                                 NaN   
6                             NaN                                 NaN   
7                             NaN                                 NaN   
8     שעות הכשרה בסמוך למינוי 26:                      שעות הכשרה 50:   
9                             NaN                                 NaN   
10                            NaN                                 NaN   
11                            NaN                                 NaN   
12                            NaN                 

In [27]:
# const results = [
#     ['Url', 'Anchor Text', 'External']
# ];
# var urls = document.getElementsByTagName('a');
# for (urlIndex in urls) {
#     const url = urls[urlIndex]
#     const externalLink = url.host !== window.location.host
#     if(url.href && url.href.indexOf('://')!==-1) results.push([url.href, url.text, externalLink]) // url.rel
# }
# const csvContent = results.map((line)=>{
#     return line.map((cell)=>{
#         if(typeof(cell)==='boolean') return cell ? 'TRUE': 'FALSE'
#         if(!cell) return ''
#         let value = cell.replace(/[\f\n\v]*\n\s*/g, "\n").replace(/[\t\f ]+/g, ' ');
#         value = value.replace(/\t/g, ' ').trim();
#         return `"${value}"`
#     }).join('\t')
# }).join("\n");
# console.log(csvContent)

In [54]:
file = "https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman2.pdf"
# tables = camelot.read_pdf(file)
# for table in tables:
#     print(table.df)
import tabula
import os
links_5778 = open("links_5778",'r',encoding='utf-8')
links=links_5778.readlines()
print(links)
# [pdf for pdf in [link.split("\t")[0] for link in links] if pdf[-4:] == ".PDF"]

FileNotFoundError: [Errno 2] No such file or directory: 'links_5778'

In [63]:
files = [doc for doc in [link.split("\t")[0].replace('"',' ')[1:-1] for link in links] if doc[-4:] == ".pdf" ][:-1]
print(files)
import requests
for i,URL in enumerate(files,start=1):
    response = requests.get(URL)
    open(str(i)+"_.pdf", "wb").write(response.content)

['https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman2.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman3.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman4.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman5.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman16.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman19.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman21.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman23.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman25.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman32.pdf', 'https://main.knesset.gov.il/Activity/committees/Women/HarassmentDocs5778/Woman35.pdf', 'https://main.knesset.gov.il/Activi

In [7]:
from win32com import client as wc
w = wc.Dispatch('Word.Application')
import glob
docs = glob.glob("5778_Docs/*")
print(docs)

['5778_Docs\\10_.doc', '5778_Docs\\11_.doc', '5778_Docs\\12_.doc', '5778_Docs\\13_.doc', '5778_Docs\\14_.doc', '5778_Docs\\15_.doc', '5778_Docs\\16_.doc', '5778_Docs\\17_.doc', '5778_Docs\\18_.doc', '5778_Docs\\19_.doc', '5778_Docs\\1_.doc', '5778_Docs\\20_.doc', '5778_Docs\\21_.doc', '5778_Docs\\22_.doc', '5778_Docs\\23_.doc', '5778_Docs\\24_.doc', '5778_Docs\\25_.doc', '5778_Docs\\26_.doc', '5778_Docs\\27_.doc', '5778_Docs\\28_.doc', '5778_Docs\\29_.doc', '5778_Docs\\2_.doc', '5778_Docs\\30_.doc', '5778_Docs\\31_.doc', '5778_Docs\\32_.doc', '5778_Docs\\33_.doc', '5778_Docs\\34_.doc', '5778_Docs\\35_.doc', '5778_Docs\\36_.doc', '5778_Docs\\37_.doc', '5778_Docs\\38_.doc', '5778_Docs\\39_.doc', '5778_Docs\\3_.doc', '5778_Docs\\40_.doc', '5778_Docs\\41_.doc', '5778_Docs\\42_.doc', '5778_Docs\\43_.doc', '5778_Docs\\44_.doc', '5778_Docs\\45_.doc', '5778_Docs\\46_.doc', '5778_Docs\\47_.doc', '5778_Docs\\48_.doc', '5778_Docs\\49_.doc', '5778_Docs\\4_.doc', '5778_Docs\\50_.doc', '5778_Docs\\5

In [62]:
os.getcwd()
# os.chdir("5778_pdf")

'C:\\Users\\turet\\Documents\\Docs\\Code_Projects\\SH_PREV_AI\\5778_pdf'

In [16]:

for i,doc in enumerate(docs,start=1):
    doc = w.Documents.Open(os.getcwd()+"/"+doc)
    doc.SaveAs(os.getcwd()+"/5778_Docx/"+str(i)+"_.docx", 16)


In [31]:
os.getcwd()

'C:\\Users\\turet\\Documents\\Docs\\Code_Projects\\SH_PREV_AI\\5778_Docx'

In [47]:
os.chdir("C:\\Users\\turet\\Documents\\Docs\\Code_Projects\\SH_PREV_AI\\5778")

In [30]:
import pandas as pd
from docx.api import Document
document = Document('2_.docx')
table = document.tables[0]
data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue
    row_data = dict(zip(keys, text))
    data.append(row_data)
    print (data)

df = pd.DataFrame(data)
df.to_csv("test.csv")

[{"המעורבים במקרה נשוא התלונה – שיוך (סטודנט, איש סגל וכו') \nללא פרטים מזהים": 'סטודנט לא מזוהה', 'מהות התלונה ופירוט המקרה – ללא פרטים מזהים': 'סטודנטית דיברה בטלפון והסטודנט עמד מאחוריה והתערב בשיחה תוך כדי אמירה מטרידה ובוטה.\nועלה במדרגות בלי שהיא הספיקה לראות מי הוא.\nהסטודנטית, שהיתה בהריון, החלה להרגיש רע בעקבות המקרה וחזרה הביתה.', 'מועד הגשת התלונה': 'אפריל 2018', 'מועד סיום הטיפול': 'מאי 2018', 'אופן הטיפול לרבות פירוט צעדי הביניים שננקטו וכן אמצעים נוספים שננקטו – כגון הגנה על המתלונן, ככל שננקטו': 'האחראית על ההטרדות ניסתה לאתר ולזהות את הסטודנט.\nהסטודנטית דיווחה שהיה סטודנט נוסף שעמד כעד ליד המדרגות.\n\nהסטודנט העד נקרא לשתי שיחות בירור אך הוא חדש במכללה ולא זיהה את הסטודנט המטריד.\n\nהסטודנטית המוטרדת קיבלה דיווח על כל צעד שנעשה בנדון.\n\nבעקבות המקרה, נשיא המכללה והאחראית על מניעת ההטרדות עברו בכל הכיתות, הקריאו את הסעיפים הרלוונטיים בלשון החוק והזהירו על הרחקה לצמיתות במקרה דומה.', 'המלצת האחראי': '', 'החלטת המוסד': '', 'החלטת הגוף המשמעתי/\nעונש, ככל שננקטו הליכי משמ

In [42]:
import pandas as pd
import io
import csv
from docx import Document

def read_docx_tables(filename, tab_id=None, **kwargs):
    """
    parse table(s) from a Word Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise
a = read_docx_tables('5_.docx')

In [44]:
a[1]

IndexError: list index out of range

In [45]:
a[0].keys()

Index(['המעורבים במקרה נשוא התלונה – שיוך (סטודנט, איש סגל וכו') \nללא פרטים מזהים',
       'מהות התלונה ופירוט המקרה – ללא פרטים מזהים', 'מועד הגשת התלונה',
       'מועד סיום הטיפול',
       'אופן הטיפול לרבות פירוט צעדי הביניים שננקטו וכן אמצעים נוספים שננקטו – כגון הגנה על המתלונן, ככל שננקטו',
       'המלצת האחראי', 'החלטת המוסד',
       'החלטת הגוף המשמעתי/\nעונש, ככל שננקטו הליכי משמעת'],
      dtype='object')

In [80]:
from pdf2docx import Converter

pdf_file = "1_.pdf"
docx_file = "1_.docx"

# convert pdf to docx
cv = Converter(pdf_file)
cv.convert(docx_file)      # all pages by default
cv.close()
# for i,file in enumerate(os.listdir(),start=1):
#     convert_pdf2docx(file,str(i)+"_.docx")

Parsing Page 1: 1/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 2: 2/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 3: 3/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 4: 4/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 5: 5/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 6: 6/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 7: 7/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 8: 8/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 9: 9/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 10: 10/11...
Ignore page due to error: 'Page' object has no attribute 'getText'
Parsing Page 11: 11/11...
Ignore page due to error: 'Page' object has no attribute 'getT

Exception: No parsed pages. Please parse page first.

In [73]:
os.listdir()


['10_.pdf',
 '11_.pdf',
 '12_.pdf',
 '13_.pdf',
 '14_.pdf',
 '15_.pdf',
 '16_.pdf',
 '17_.pdf',
 '18_.pdf',
 '19_.pdf',
 '1_.pdf',
 '20_.pdf',
 '21_.pdf',
 '22_.pdf',
 '23_.pdf',
 '24_.pdf',
 '25_.pdf',
 '26_.pdf',
 '27_.pdf',
 '28_.pdf',
 '2_.pdf',
 '3_.pdf',
 '4_.pdf',
 '5_.pdf',
 '6_.pdf',
 '7_.pdf',
 '8_.pdf',
 '9_.pdf']

In [ ]:
import aspose.words as aw

fileNames = [ "Input1.pdf", "Input2.pdf" ]

output = aw.Document()
# Remove all content from the destination document before appending.
output.remove_all_children()

for fileName in fileNames:
    input = aw.Document(fileName)
    # Append the source document to the end of the destination document.
    output.append_document(input, aw.ImportFormatMode.KEEP_SOURCE_FORMATTING)

output.save("Output.docx");

In [65]:
os.listdir()

['10_.pdf',
 '11_.pdf',
 '12_.pdf',
 '13_.pdf',
 '14_.pdf',
 '15_.pdf',
 '16_.pdf',
 '17_.pdf',
 '18_.pdf',
 '19_.pdf',
 '1_.pdf',
 '20_.pdf',
 '21_.pdf',
 '22_.pdf',
 '23_.pdf',
 '24_.pdf',
 '25_.pdf',
 '26_.pdf',
 '27_.pdf',
 '28_.pdf',
 '2_.pdf',
 '3_.pdf',
 '4_.pdf',
 '5_.pdf',
 '6_.pdf',
 '7_.pdf',
 '8_.pdf',
 '9_.pdf']